## 1. 스파크 클러스터 접속

Spark Context 생성시켜 `sc`변수에 지정하고 이를 연결지점으로 스파크 클러스터에 접속한다.

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

sc = SparkContext('local','abstract_data')  # if using locally
spark = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=abstract_data, master=local) created by __init__ at <ipython-input-1-b0ecd9dad820>:8 

## 2. 데이터프레임 생성
### 2.1. 리스트 &rarr; 스파크 데이터프레임으로 변환

튜플 리스트를 `spark.createDataFrame` 함수로 스파크 데이터프레임을 생성시킨다.

In [ ]:
# 튜플 리스트
party_list = [('민주당',1), ('바른미래',2), ('자유한국',3), ('민주당',30), ('바른미래',20), ('자유한국',10)]
# 튜플리스트에서 RDD생성 
party_rdd = sc.parallelize(party_list)

# 스파크 데이터프레임 생성
party_df = spark.createDataFrame(party_rdd, schema=['정당', '득표수'])

# Check the type of people_df
print("party_df 자료형: ", type(party_df))

party_df.show()

### 2.2. `.csv` 파일을 스파크 데이터프레임으로 변환
`iris.csv` 파일을 스파크 데이터프레임으로 변환시킨다. 이를 위해서 `.read.csv()` 함수를 사용하고 `.printSchema()`메쏘드로 데이터프레임 변수별 자료형을 확인할 수 있다.

In [ ]:
# iris.csv 파일에서 생성
iris_df = spark.read.csv("../data/iris.csv", header=True, inferSchema=True)

# iris_df 스키마
iris_df.printSchema()

## 3. 스파크 데이터프레임 변환작업
외부 데이터를 가져온 후에 다양한 메쏘드를 활용하여 데이터프레임 변환작업을 수행한다.

In [ ]:
# 데이터프레임 일반현황
print("데이터프레임 행수: {}".format(iris_df.count()), "\n데이터프레임 열수: {}".format(len(iris_df.columns)), "\n변수명: ", iris_df.columns)
# 데이터프레임 변수 및 행 필터링
iris_subset_df = iris_df.select('sepal_length', 'petal_length', 'species').filter(iris_df.species =="setosa")
# 첫 관측점 10개만 추출
iris_subset_df.show(10)

## 4. 스파크 데이터프레임 SQL 변환작업
외부 데이터를 가져온 후에 SQL 구문을 활용하여 데이터프레임에 대한 변환작업을 수행할 수 있다.

In [ ]:
# iris 테이블 생성 - `iris_df` 스파크 데이터프레임을 콕 집음.
iris_df.createOrReplaceTempView("iris")

# 쿼리를 생성
iris_query = '''SELECT sepal_length, petal_length, species 
                FROM iris 
                WHERE species == "setosa"'''

# 쿼리를 실행시켜 스파크 데이터프레임 객체 신규 생성
iris_query_df = spark.sql(iris_query)

# 스파크 데이터프레임 결과 출력
iris_query_df.show(10)

## 5. 데이터프레임 시각화
시각화를 위해서 스파크 데이터프레임을 판다스 데이터프레임으로 변환시켜야 된다. 이를 위해서 `.toPandas()` 함수를 사용하고 `matplotlib.pyplot`을 사용해서 시각화한다.

In [ ]:
import matplotlib.pyplot as plt

iris_query_pd = iris_query_df.toPandas()

# 시각화
iris_query_pd.plot(kind='density')
plt.show()